# Lesson 30: JavaScript for stand-alone Bokeh apps

<hr>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import bokeh.plotting
import bokeh.io

bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

As we have seen when [building dashboards](l29_dashboards.ipynb), we can build remarkable interactivity in our data displays, allowing for low-effort exploration of data sets. Panel, with Bokeh output, allow us to write Python code that can update our displays as we adjust parameters or select variables. We can also serve up a notebook to get only the data displays we like on their own browser tabs. I hope you recognize how powerful this is for exploring your data sets.

There is a major drawback to this approach, though. In order to interactively explore using the dashboard, you need to have a Python interpreter running, either in the Jupyter notebook or when you serve your dashboards using `panel serve` on the command line. If you wanted to include an interactive plot in a publication, it would be nice to be able to interact with the plots/dashboard directly in the browser without the reader having to launch a Jupyter notebook or serve it up on their local machine (and have all of the necessary installations). In some cases, you will just need the Python interpreter for sophisticated dashboards that do involved calculations. But in many cases, with just a little JavaScript, you can make beautiful interactive graphics rendered in pure HTML with JavaScript.

## JavaScript and Bokeh

We have been using Bokeh for rendering out plots and dashboards. (Even though we used a Panel interface, the dashboards we have made are rendered with Bokeh.) As we have seen, Bokeh is a Python library for generating graphics that can be visualized and interacted with in the browser. In order to display the graphics and enable the interactions, Bokeh relies on its client-side [JavaScript](https://en.wikipedia.org/wiki/JavaScript) library, [BokehJS](https://docs.bokeh.org/en/latest/docs/dev_guide/bokehjs.html). It is this JavaScript library that handles all of the rendering, zooming, selecting, etc., as well has the widgets. The **callbacks**, which are functions that get executed to update information about the plot based on changes in the widgets (such as computing the values of a smooth curve, as we saw in our exploration of the theoretical model for fold change in gene expression, or selecting pieces of a Pandas data frame, as we saw in the dashboard for the facial recognition data), are in Python. This is what requires the Python interpreter to be running.

Bokeh allows for pure JavaScript callbacks. And it turns out that if you know just a little JavaScript, you can make your dashboards have full functionality standing alone in the browser without Python running.

So, in this lesson, we will learn a little JavaScript to make our dashboards be totally portable. We will not delve into the details of the language (that would require a complete bootcamp!), but will just give a little syntax. We will do it all in the context of making interactive plots. As you will see, a little JavaScript goes a long way!

## Using ColumnDataSources

Let's look at how we have been building plots with Bokeh so far. To make the plot, I'll first generate some random data.

In [2]:
rg = np.random.default_rng()
x = np.linspace(-3, 3, 50)
y = rg.normal(x, 1)

Now, I'll make a figure.

In [3]:
p = bokeh.plotting.figure(
    frame_width=250,
    frame_height=250,
    x_axis_label='x',
    y_axis_label='y',
)

Then, when it becomes time to populate glyphs, we specify arrays (or lists or columns of data frames) for `x` and `y`.

In [4]:
p.circle(x, y)

bokeh.io.show(p)

Let's now peak under the hood a bit. When we called `p.circle()`, we created a **renderer**, which populates the plot with glyphs. A plot has a list of renderers, which we can access with `p.renderes`.

In [5]:
p.renderers

[GlyphRenderer(id='1037', ...)]

This is a list of `GlyphRenderer` instances. Each `GlyphRenderer` has a `data_source` attribute, which contains the data that dictate where the glyphs are placed. Let's look at the `data_source` for the plot we have just made.

In [6]:
p.renderers[0].data_source

ColumnDataSource(id='1034', ...)

This is a `ColumnDataSource` instance. This Bokeh data type is at the heart of many of the plots you can make with Boken and is an object that we can interact directly with using JavaScript. Importantly, the `ColumnDataSource` has a `data` attribute that contains the data for the plot stored as a dictionary.

In [7]:
p.renderers[0].data_source.data

{'x': array([-3.        , -2.87755102, -2.75510204, -2.63265306, -2.51020408,
        -2.3877551 , -2.26530612, -2.14285714, -2.02040816, -1.89795918,
        -1.7755102 , -1.65306122, -1.53061224, -1.40816327, -1.28571429,
        -1.16326531, -1.04081633, -0.91836735, -0.79591837, -0.67346939,
        -0.55102041, -0.42857143, -0.30612245, -0.18367347, -0.06122449,
         0.06122449,  0.18367347,  0.30612245,  0.42857143,  0.55102041,
         0.67346939,  0.79591837,  0.91836735,  1.04081633,  1.16326531,
         1.28571429,  1.40816327,  1.53061224,  1.65306122,  1.7755102 ,
         1.89795918,  2.02040816,  2.14285714,  2.26530612,  2.3877551 ,
         2.51020408,  2.63265306,  2.75510204,  2.87755102,  3.        ]),
 'y': array([-3.46739023, -1.97774081, -3.57512495, -3.29507437, -3.90105068,
        -2.12918742, -2.95419596, -2.50317252, -0.5578926 , -2.49863993,
        -1.12896566, -2.95414326, -2.06574041, -1.64050674, -0.40250839,
        -1.39187748, -1.70267032, -0.87

Now that we know how the data are represented under the hood in a Bokeh plot and have an understanding of how `ColumnDataSource`s work, we can instead create a plot directly using the `ColumnDataSource`. When a glyph method like `p.circle()` is called, if we use a `source` kwarg, we can specify a `ColumnDataSource` explicitly, giving us access to its contents. To instantiate a `ColumnDataSource`, we call `bokeh.models.ColumnDataSource` with a dictionary (or Pandas data frame) argument that will be used for the data.

In [8]:
p = bokeh.plotting.figure(
    frame_width=250,
    frame_height=250,
    x_axis_label='x',
    y_axis_label='y',
)

# Make the ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(x=x, y=y))

# Populate glyphs
p.circle(source=cds, x="x", y="y")

bokeh.io.show(p)

Note that when we use the `source` kwarg, we specify the `x` and `y` values as keys of the dictionary containing the data.

One of the beautiful properties about a `ColumnDataSource` is that we can update its data and the plot will also update.

In [9]:
# Change the y-data
cds.data['y'] = np.exp(np.sin(x))

# Reshow the plot
bokeh.io.show(p)

The `ColumnDataSource` can also be directly accessed in JavaScript callbacks. This is what will enable us to change the data in a plot in response to changes in widgets.

## Example 1: Exploring the Normal pdf

In our [lesson on dashboarding](l29_dashboards.ipynb), we build an interactive plot of the probability density function (pdf) for the Normal distribution. We used the `scipy.stats` package to get the mathematical expression for the pdf. Since we will code it up in JavaScript, I write it here.

\begin{align}
f(x;\mu, \sigma) = \frac{\mathrm{e}^{-(x-\mu)^2/2\sigma^2}}{\sqrt{2\pi\sigma^2}}.
\end{align}

I will not recreate the little dashboard we made in the dashboarding lesson for visualizing the Normal pdf, but so that it can stand alone. As a reminder, here is the code we used before to create the dashboard.

```python
opts = dict(show_grid=True, frame_height=200, frame_width=350, color="#1f77b3")

mu_slider = pn.widgets.FloatSlider(
    name="µ", start=-5, end=5, step=0.1, value=0
)
sigma_slider = pn.widgets.FloatSlider(
    name="σ", start=0.1, end=5, step=0.1, value=1
)


@pn.depends(mu_slider.param.value, sigma_slider.param.value)
def plot_normal_pdf(mu=0, sigma=1):
    x = np.linspace(-10, 10, 200)
    y = scipy.stats.norm.pdf(x, loc=mu, scale=sigma)

    return hv.Curve(data=(x, y), kdims=["x"], vdims=["f(x ; μ, σ)"]).opts(
        **opts
    )


widgets = pn.Column(
    pn.Spacer(height=30),
    mu_slider,
    pn.Spacer(height=15),
    sigma_slider,
    width=200,
)

pn.Row(plot_normal_pdf, pn.Spacer(width=15), widgets)
```

Now, let's build the dashboard with a JavaScript callback so it can stand alone with a Python interpreter running. First, the setup of the setup of the sliders, data, and figure are the same as we would normally do with Bokeh (not HoloViews).

In [10]:
# Initial setup of the sliders is the same
mu_slider = bokeh.models.Slider(title="µ", start=-5, end=5, step=0.1, value=0)
sigma_slider = bokeh.models.Slider(title="σ", start=0.1, end=5, step=0.1, value=1)

# Setup of x and y values is also the same
x = np.linspace(-10, 10, 200)
pdf = scipy.stats.norm.pdf(x, loc=0, scale=1)

# Set up the plot as a Bokeh figure
p = bokeh.plotting.figure(
    frame_width=300,
    frame_height=200,
    x_axis_label="x",
    y_axis_label="f(x; μ, σ)",
    x_range=[-10, 10],
)

Notice that we are using sliders instantiated from Bokeh as opposed to Panel. That is, we use `bokeh.models.Slider()` instead of `pn.widgets.FloatSlider()`. (Note that the `name` kwarg for a Panel slider is `title` for a Bokeh slider; otherwise the syntax for their instantiation is the same.) I did this because when we render stand-alone HTML pages, there can be some problems with updating axis ranges when updated data in the stand-along HTML app. It's a bug in Panel that will likely be fixed soon.

Next, we set up a `ColumnDataSource` when we build the plot so we have access to it when manipulate the sliders and use a JavaScript callback.

In [11]:
# Use a ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(x=x, pdf=pdf))
p.line(source=cds, x="x", y="pdf", line_width=2);

Laying out the sliders and plot is the same as before, except we will use Bokeh's native layout system instead of Panel's. They really only differ slightly.

In [12]:
# Laying out the widgets is the same
widgets = bokeh.layouts.column(
    bokeh.layouts.Spacer(height=30),
    mu_slider,
    bokeh.layouts.Spacer(height=15),
    sigma_slider,
    width=200,
)

# Lay out with the plot and widgets
layout = bokeh.layouts.row(p, bokeh.layouts.Spacer(width=15), widgets)

Now, we need to write JavaScript code for the callback. Whenever a slider changes value, we want to recompute the values of the Normal pdf. Here is JavaScript code to do that.

```javascript
function normalPDF(x, mu, sigma) {
    var expArg = (x - mu) ** 2 / 2.0 / sigma ** 2;
    return Math.exp(-expArg) / sigma / Math.sqrt(2 * Math.PI);
}

// For convenience, get views into the ColumnDataSource data
var x = cds.data['x'];
var pdf = cds.data['pdf'];

// Pull the values off of the sliders
var mu = mu_slider.value;
var sigma = sigma_slider.value;

// Loop through and update
var xLen = x.length;
for (var i = 0; i < xLen; i++) {
    pdf[i] = normalPDF(x[i], mu, sigma);
}

// Once the ColumnDataSource is changed, we need to use its 
// .change.emit() method to update the plot
cds.change.emit();
```

Let's discuss the syntax. First, comments in JavaScript are preceded with `//`. Functions in JavaScript are declared like this:

```javascript
function funcName() {
    // function body
}
```

The `function` keyword is followed by the function name, with its arguments in parentheses separated by commas. The function body is then contained between braces.

Within the `normalPDF()` function, we make some calculations. First, we define a variable that is the argument for the exponentiation, $-(x-\mu)^2/2\sigma^2$. Note that whenever we use a new variable, it must be preceded by the keyword `var` (or `let`, but we will not get into the differences between those keywords). The arithmetic operators, including raise-to-power, are the same as in Python. We end each statement with a semicolon.

Next comes the `return` statement. Its syntax is just like Python. only difference in the return statement is that we need to use JavaScript's `Math` module to get access to the exponentiation and square root functions, and also to the value of π. Note that this function is _not_ vectorized. It assumes `x`, `mu`, and `sigma` are scalar valued and computes the value of the Normal pdf at that given value of `x`.

Conveniently, accessing the `ColumnDataSource.data` attribute using JavaScript is exactly as in Python. It is indexed just like you would index a dictionary in Python. For convenience in the above code, we create views into the `ColumnDataSource` using `var x = cds.data['x'];` and `var pdf = cds.data['pdf'];`.

Next, we pull the values from the sliders. This is also done as in Python (except with the necessary `var`).

Now that we have access to the data and the slider values, we need to loop through the data in the `ColumnDataSource` and update it according to the values of the sliders. A `for` loop in JavaScript typically has the following form.

```javascript
for (var i = 0; i < lastValueOfiYouWant; i++) {
    // body of for loop
}
```

Inside the parenthetical after the `for` keyword are two semicolons. Before the first is an initialization. After the first semicolon is a conditional statement. The loop will continue so long as that conditional statement gives `true` (in JavaScript the keyword for truth is all lowercase, unlike in Python). Finally, after the second semicolon is the iteration statement. It is executed whenever a trip through the body of the loop concludes. The syntax `i++` is equivalent to Python's `i += 1`.

So, the `for` loop in the above statement loops through each x-value in the plot and updates the value of the Normal pdf using the parameter values we have pulled off of the sliders.

Finally, in order for the plot to update, you need to execute the `cds.change.emit()` function, which pushes the updates of the `ColumnDataSource` to the rendered plot.

This code is in JavaScript, so obviously the Python interpreter will not know what to do with it. It is not really used until things get handed off to the BokehJS library. So, we need to store the code as a Python string.

In [13]:
# JavaScript code for the callback stored as a string
jscode = """
function normalPDF(x, mu, sigma) {
    var expArg = (x - mu) ** 2 / 2.0 / sigma ** 2;
    return Math.exp(-expArg) / sigma / Math.sqrt(2 * Math.PI);
}

// For convenience, get views into the ColumnDataSource data
var x = cds.data['x'];
var pdf = cds.data['pdf'];

// Pull the values off of the sliders
var mu = mu_slider.value;
var sigma = sigma_slider.value;

// Loop through and update
var xLen = x.length;
for (var i = 0; i < xLen; i++) {
    pdf[i] = normalPDF(x[i], mu, sigma);
}

// Once the ColumnDataSource is changed, we need to use its 
// .change.emit() method to update the plot
cds.change.emit();
"""

Now that we have the code, we need to connect it to the sliders. That is, we need to let the slider objects know that when they change, the JavaScript code should be triggered.

In [14]:
args = dict(cds=cds, mu_slider=mu_slider, sigma_slider=sigma_slider)

# Connect the callback to the sliders
mu_slider.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
sigma_slider.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))

The `js_on_change()` method links the sliders to the JavaScript code. The first argument of the `js_on_change()` method is what is changing that will trigger the callback. In this case, we want the callback to be triggered whenever the value of the slider changes. The second argument is a Bokeh `CustomJS` object. This is constructed by specifying the code as a string and the arguments that will be passed to the JavaScript code (in this case the `ColumnDataSource` and the two sliders).

Note that if we used Panel-based sliders instead of the Bokeh sliders we are using here, the syntax is

```python
slider_created_using_panel.jscallback(value=jscode, args=args)
```

Now we are ready to check out our layout!

In [15]:
# Display the layout
bokeh.io.show(layout)

This is more or less the same about of work we did creating the dashboard with Panel except:

1. We had to use Bokeh to build the sliders and layout (we didn't _have_ to, but it avoids some Panel bugs I've bumped in to recently)
2. We used Bokeh instead of HoloViews to build the plot.
3. We used a `ColumnDataSource` to do so.
4. The callbacks were written in JavaScript and linked using `js_on_change()` instead of Panel decorators.

That last item is the challenging one. But, I hope you can see that it's not _too_ bad to code up the JavaScript. The payoff is big. You get a stand-alone interactive plot that _anyone_ can view and play with in their browser without even having Python installed.

Let's now make that HTML document.

In [16]:
bokeh.io.save(layout, "normal_pdf_app.html")

/Users/bois/opt/anaconda3/lib/python3.8/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/bois/opt/anaconda3/lib/python3.8/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/bois/Dropbox/git/programming_bootcamp/2021/content/lessons/normal_pdf_app.html'

We can again ignore the warnings. We now have our stand-alone app!

(Note that if you wanted to save a Panel layout to HTML, the syntax is `layout_made_with_panel.save("filename.html")`.

## Exploring a data set

In the [dashboarding lesson](l29_dashboards.ipynb), we used data from the [Beattie, et al.](https://doi.org/10.1098/rsos.160321) study of how sleep deprivation affects facial matching ability to build a dashboard to explore a data set. We set up a grid matrix of plots in that example. Here, I will demonstrate how to build a simpler dashboard, allowing for a single plot of one variable (e.g., percent correct facial discrimination) versus another (e.g. sleep score). The variables to be plotted can be selected, as can the categorical variable with which to color the glyphs. All interactions are with JavaScript callbacks, so such a plot could appear in a publication with full interactivity for all readers.

Of course, we'll start with loading in the data.

In [17]:
df = pd.read_csv('data/gfmt_sleep.csv', na_values='*')

# Add column for insomnia
df['insomnia'] = df['sci'] <= 16

df.head()

participant number gender  age  correct hit percentage  \
0                   8      f   39                      65   
1                  16      m   42                      90   
2                  18      f   31                      90   
3                  22      f   35                     100   
4                  27      f   74                      60   

   correct reject percentage  percent correct  confidence when correct hit  \
0                         80             72.5                         91.0   
1                         90             90.0                         75.5   
2                         95             92.5                         89.5   
3                         75             87.5                         89.5   
4                         65             62.5                         68.5   

   confidence when incorrect hit  confidence when correct reject  \
0                           90.0                            93.0   
1                           55.5                            70.5   
2                           90.0                            86.0   
3                            NaN                            71.0   
4                           49.0                            61.0   

   confidence when incorrect reject  confidence when correct  \
0                              83.5                     93.0   
1                              50.0                     75.0   
2                              81.0                     89.0   
3                              80.0                     88.0   
4                              49.0                     65.0   

   confidence when incorrect  sci  psqi  ess  insomnia  
0                       90.0    9    13    2      True  
1                       50.0    4    11    7      True  
2                       88.0   10     9    3      True  
3                       80.0   13     8   20      True  
4                       49.0   13     9   12      True

To build the dashboard, we will start by setting up the selectors, one for the x-axis variable, one for the y-axis variable, and one for how we will color the glyphs.

In [18]:
# Options for x- and y- selector; omit part. num., gender, and insomnia
xy_options = list(
    df.columns[~df.columns.isin(["participant number", "gender", "insomnia"])]
)

x_selector = bokeh.models.Select(
    title="x", options=xy_options, value="percent correct", width=200,
)

y_selector = bokeh.models.Select(
    title="y", options=xy_options, value="confidence when correct", width=200,
)

colorby_selector = bokeh.models.Select(
    title="color by", options=["none", "gender", "insomnia",], value="none", width=200,
)

Next, we can set up the `ColumnDataSource`. In a dashboard like this, I find it is easier to set up a `ColumnDataSource` containing the columns that are used in rendering the plot (that is, only three columns, the x-values, the y-values, and the color-values). The color column is a column containing strings representing the colors. We'll start with a plot of confidence when correct versus percent correct with no additional coloring.

In [19]:
render_cds = bokeh.models.ColumnDataSource(
    {
        "x": df["percent correct"],
        "y": df["confidence when correct"],
        "color": ["#1f77b3"] * len(df),
    }
)

Now that we have the `ColumnDataSource` for rendering, we need to convert the data frame `df` into a `ColumnDataSource` so we can have access to is using JavaScript.

In [20]:
cds = bokeh.models.ColumnDataSource(df)

Now we can make our figure!

In [21]:
p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=250,
    x_axis_label="percent correct",
    y_axis_label="confidence when correct",
)

circle = p.circle(
    source=render_cds, x="x", y="y", fill_color="color", line_color="color",
)

Finally, we write our JavaScript code for the callbacks. These will happen whenever the selectors change. Following is the JavaScript code.

```javascript
// Extract what we want to color by from selector
var colorby = colorby_selector.value;

// View of the colors for convenience
var colors = render_cds.data['color'];

// Convenient to have the number of data points
var n = colors.length;

// Update rendered data from full data set
render_cds.data['x'] = cds.data[x_selector.value];
render_cds.data['y'] = cds.data[y_selector.value];

// Update colors
if (colorby === 'none') {
    for (var i = 0; i < n; i++) {
        colors[i] = '#1f77b3';
    }
}
else if (colorby === 'gender') {
    for (var i = 0; i < n; i++) {
        if (cds.data['gender'][i] === 'f') {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}
else if (colorby === 'insomnia') {
    for (var i = 0; i < n; i++) {
        if (cds.data['insomnia'][i]) {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}

// Update axis labels to reflect what was selected
xaxis[0].axis_label = x_selector.value;
yaxis[0].axis_label = y_selector.value;

render_cds.change.emit();
```

Except for the `if` statements, which have new syntax, the code should be self-explanatory. There is the added wrinkle of the `xaxis` and `yaxis` variables, which we will have to pass into the JavaScript code as arguments. These are Bokeh `LinearAxis` objects that we need to access to update the axis labels for what we are plotting.

In JavaScript, an `if`-`else` statement has the structure

```c
if (statement_returning_boolean) {
    // code to be executed if statement_returning_boolean is true
}
else if (another_statement_returning_boolean) {
    // code to be executed if another_statement_returning_boolean is true
}
else {
    // code to be executed if all booleans were false
}
```

Note the use of `===` for the equals comparison operator. In JavaScript, if I use `a == b`, JavaScript tries to do type comparisons to compare things. `0 == ''` will evaluate `true`. The `===` operator means that there are no type comparisons. So, `0 === ''` evaluates `false`.

Note that we have our JavaScript code, let's put it to use!

In [22]:
jscode = """
// Extract what we want to color by from selector
var colorby = colorby_selector.value;

// View of the colors for convenience
var colors = render_cds.data['color'];

// Convenient to have the number of data points
var n = colors.length;

// New data
render_cds.data['x'] = cds.data[x_selector.value];
render_cds.data['y'] = cds.data[y_selector.value];

// Update colors
if (colorby === 'none') {
    for (var i = 0; i < n; i++) {
        colors[i] = '#1f77b3';
    }
}
else if (colorby === 'gender') {
    for (var i = 0; i < n; i++) {
        if (cds.data['gender'][i] === 'f') {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}
else if (colorby === 'insomnia') {
    for (var i = 0; i < n; i++) {
        if (cds.data['insomnia'][i]) {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}

// Update axis labels to reflect what was selected
xaxis[0].axis_label = x_selector.value;
yaxis[0].axis_label = y_selector.value;

render_cds.change.emit();
"""

args = dict(
    render_cds=render_cds,
    cds=cds,
    colorby_selector=colorby_selector,
    x_selector=x_selector,
    y_selector=y_selector,
    xaxis=p.xaxis,
    yaxis=p.yaxis,
)

Note that I have put both data sources, the selectors, and the `p.axis` and `p.yaxis` objects as arguments to the JavaScript code. Now we can link the callbacks.

In [23]:
colorby_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
x_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
y_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))

And finally, let's lay it out!

In [24]:
layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

bokeh.io.show(layout)

Beautiful! We can again save the result and have a stand-alone interactive HTML file.

In [25]:
bokeh.io.save(layout, 'sleep_facial_recognition.html')

/Users/bois/opt/anaconda3/lib/python3.8/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/bois/opt/anaconda3/lib/python3.8/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/bois/Dropbox/git/programming_bootcamp/2021/content/lessons/sleep_facial_recognition.html'

## Conclusions

I hope you can see that a little JavaScript goes a long way! But knowing just some basic JavaScript syntax, you can build totally sharable, browser-display-and-playable data displays.

## Computing environment

In [26]:
%load_ext watermark
%watermark -v -p numpy,pandas,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.22.0

numpy     : 1.20.2
pandas    : 1.2.4
bokeh     : 2.3.2
jupyterlab: 3.0.14

